In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [25]:
df = pd.read_csv('preprocessing.csv')

In [26]:
pd.set_option('display.max_columns', None) # чтобы увидеть все столбцы

In [27]:
def count_bathrooms(bathroom_string):
    if pd.isna(bathroom_string):
        return 0
    bathroom_string = str(bathroom_string)
    numbers = re.findall(r'\((\d+)\)', bathroom_string)
    return sum(map(int, numbers)) if numbers else 0

sanuzels = df['Санузел']
df['Общее количество санузлов'] = sanuzels.apply(count_bathrooms)

# находим моду для количества санузлов
mode_bathrooms = df['Общее количество санузлов'].mode().iloc[0]

# заменяем нулевые значения (которые были NaN) на моду
df['Общее количество санузлов'] = df['Общее количество санузлов'].replace(0, mode_bathrooms)

In [28]:
# Бинарный признак: центр или нет
df['is_center'] = df['Адрес'].apply(lambda x: 1 if 'центр' in x.lower() else 0)

In [29]:
# удаляем: Unnamed: 0 - дублирует индексы, Тип - у всех одинаковый, Санузел и Адрес - взяли что нужно, Телефоны и Ссылка на объявление - не влияют на цену, 
# удаляем: Дом, Площадь комнат, м2 - смотрим только на общую площадь, Название ЖК и Серия дома - много NaN
df = df.drop(columns=['Unnamed: 0', 'Тип', 'Санузел', 'Адрес', 'Телефоны', 'Дом', 'Площадь комнат, м2', 'Название ЖК', 'Серия дома', 'Ссылка на объявление'])

In [30]:
# извлекаем только кол-во комнат, а NaN заменяем на 0
df['Количество комнат'] = df['Количество комнат'].str.extract(r'(\d+)').fillna(0).astype(int)

In [31]:
# извлекаем только расстояние до метро, а NaN заменяем на 60, т.к. где не указано - скорее всего далеко
df['Метро'] = df['Метро'].str.extract(r'(\d+)')[0].fillna(60).astype(int)

In [32]:
# преобразовать в бинарный признак (1 — есть парковка, 0 — нет). NaN заменить на 0.
df['Парковка'] = df['Парковка'].notna().astype(int)

In [33]:
# Извлечение признаков из описания, которые могут влиять на стоимость аренды
df['has_furniture'] = df['Описание'].apply(lambda x: 1 if 'мебель' in str(x).lower() else 0)
df['has_washer'] = df['Описание'].apply(lambda x: 1 if 'стиральная машина' in str(x).lower() else 0)

In [34]:
# просто длина описания, чем больше содежание, тем выше цена
df['Описание'] = df['Описание'].str.len().fillna(0)

In [35]:
# кодиовка ремонта, а где нет - 0
repair_mapping = {
    'Косметический': 1,
    'Евроремонт': 2,
    'Дизайнерский': 3
}
df['Ремонт'] = df['Ремонт'].map(repair_mapping).fillna(0)

In [36]:
# извлечь количество балконов, NaN заменить на 0.
df['Балкон'] = df['Балкон'].str.extract(r'(\d+)').fillna(0).astype(int)

In [37]:
# кодировка по окнам
windows_mapping = {
    'Во двор': 1,
    'На улицу': 2,
    'На улицу и двор': 3
}
df['Окна'] = df['Окна'].map(windows_mapping)
# определение моды среди непустых значений
mode_windows = df['Окна'].mode().iloc[0]
# замена пустых значений на моду
df['Окна'] = df['Окна'].fillna(mode_windows)

In [38]:
# разделение колонки на две с последующим удалением
df['Можно с детьми'] = df['Можно с детьми/животными'].str.contains('Можно с детьми').fillna(0).astype(int)
df['Можно с животными'] = df['Можно с детьми/животными'].str.contains('Можно с животными').fillna(0).astype(int)
df = df.drop(columns=['Можно с детьми/животными'])

In [39]:
# замена на длину содержания
df['Дополнительно'] = df['Дополнительно'].str.split(',').str.len().fillna(0)

In [40]:
# Функция для перевода высоты из см в м
def convert_to_meters(value):
    if value > 10:
        return value / 100
    return value

In [41]:
# Высота потолков: исправление аномалий
df['Высота потолков, м'] = df['Высота потолков, м'].apply(convert_to_meters)
# Замена NaN на среднее значение
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(df['Высота потолков, м'].mean())

In [42]:
# закодировать наличие лифта (1 — есть, 0 — нет). NaN заменить на 0.
df['Лифт'] = df['Лифт'].notna().astype(int)

In [43]:
# закодировать наличие мусоропровода (1 — есть, 0 — нет). NaN заменить на 0.
df['Мусоропровод'] = df['Мусоропровод'].eq('Да').astype(int)

In [44]:
# проверка на отсутсвие NaN
df.isnull().sum() == 0

ID  объявления               True
Количество комнат            True
Метро                        True
Площадь, м2                  True
Парковка                     True
Цена                         True
Описание                     True
Ремонт                       True
Балкон                       True
Окна                         True
Дополнительно                True
Высота потолков, м           True
Лифт                         True
Мусоропровод                 True
Общее количество санузлов    True
is_center                    True
has_furniture                True
has_washer                   True
Можно с детьми               True
Можно с животными            True
dtype: bool

In [45]:
# словарь для переименования колонок
rename_dict = {
    'ID  объявления': 'ad_id',
    'Количество комнат': 'rooms',
    'Метро': 'metro',
    'Площадь, м2': 'area_sqm',
    'Парковка': 'parking',
    'Цена': 'target',
    'Описание': 'description',
    'Ремонт': 'renovation',
    'Балкон': 'balcony',
    'Окна': 'windows',
    'Дополнительно': 'additional',
    'Высота потолков, м': 'ceiling_height_m',
    'Лифт': 'elevator',
    'Мусоропровод': 'garbage_chute',
    'Общее количество санузлов': 'total_bathrooms',
    'is_center': 'is_center',
    'has_furniture': 'has_furniture',
    'has_washer': 'has_washer',
    'Можно с детьми': 'children_allowed',
    'Можно с животными': 'pets_allowed'
}

# переименование колонок
df = df.rename(columns=rename_dict)

In [51]:
# перемещение 'target' в конец
target_col = df.pop('target')
df['target'] = target_col 

In [52]:
df.to_csv('data.csv', index=False)